Coursera의 Neural Networks for Machine Learning 강의를 정리한 내용임.   
2016.12.19. by Dongwan Kim

# Week11. Hopfield nets and Boltzmann machines

### Lecture 11a, Hopfield Nets

<u>Hopfield Nets</u>

**A Hopfield net** is composed of <u>binary threshold units</u> with <u>recurrent connections</u> between them.

우선 **binary threshold unit**(McCulloch-Pitts neuron)은 week1에서 배웠다시피,    
입력값($x_i$)들의 가중합(weighted sum, $\sum_{i} x_i w_i$)이 임계치(threshold, $\theta$)를 넘을 경우 1을 출력하고 그렇지 않을 경우 0을 는 함수(step function)이다. (activation function, $\phi(\cdot)$로 step function을 사용하여 출력값을 1과 -1로 사용하는 경우도 있다.)

$$z = \xi(x, w) = \sum_{i} x_i w_i$$   
$$y = \phi(z) = \begin{cases} 1 & \rm{if} ~ z \geq \theta \\ 0 & \rm{otherwise}
\end{cases}$$


※ References
- https://en.wikipedia.org/wiki/Artificial_neuron
- Charu C. Aggawal, Data Classification: Algorithms and Applications, Chapter 8.2.2

<img  src="./_images/01_binary_threshold_neurons1.PNG"/>

다음으로 **recurrent connection**은 week7에서 배웠다시피,     
unit간의 연결이 <a href="https://en.wikipedia.org/wiki/Directed_graph">directed cycle</a>형태인 neural network이다.

<img  src="./_images/07_RNN_ffnn.PNG"/>

즉 **A Hopfield net**은 binary input을 받아 binary value를 출력하는 unit들이 directed cycle 형태로 연결된 neural network라는 것이다.

일반적으로 non-linear unit들로 구성된 RNN은 그 특성을 분석하기가 쉽지 않은데, connection들이 symmetric할 경우 network를 **global energy function**으로 모델링 할 수 있음을 밝혀 냈다.(John Hopfield) 여기서 symmetric이라는 것은 $w_{ij}$와 $w_{ji}$가 같다는 것이고 directed cycle이므로 $w_{ii} = 0$이다. (따라서 hopfield net에서의 weight matrix는 symmetric하고 digonal term이 0)

여타의 머신러닝 알고리즘의 학습 과정이 cost function을 최소화 하는 혹은 $p(W|D)$(posterior probability of weights given data)를 최대화 하는 weight들을 구하는 것인 것과 마찬가지로, Hopfield network에서는 **global energy function**라는 것을 정의하고 이 함수의 값이 최소가 되는 parameter값들(configuration)을 찾는다.(그 표현이 다를 뿐 결국은 같은 결과로 귀결된다.)

<u>The energy function</u>

상태(state)값이 $s_i$인 neuron $n_i$와 상태값이 $s_j$인 neuron $n_j$가 connection weight $w_{ij}$으로 연결되어 있고 neuron $n_i$에 부여된 bias가 $b_i$일 때, global energy function은 아래와 같다.

$$E = -\sum_i s_i b_i - \sum_{i < j} s_i s_j w_{ij}$$

<img  src="./_images/11_hopfield_ex4_new.PNG"/>

예를들어 위와 같은 hopfield net이 주어졌을 때, (bias term $b_i$는 없다고 가정) global energy는 5(=3+3-1)가 된다. (각 neuron의 state값이 1혹은 0이므로 서로 연결된 neuron pair의 state 값이 모두 1일인 $w_{ij}$들만 더하면 된다.)

이렇게 energy funciton에 대해 알아봤고, energy라는 용어에서 알 수 있는 것 처럼 이 함수는 그 입력값에 따라 어떤 안정된 상태 값으로 회귀하는 성질을 갖는데, 안정된 상태로 회귀 한다는 것은 global energy가 작아지도록 neuron들의 상태값($s_i$)들을 변화시키다 보면 global energy가 최소화어 더 이상 neuron들의 상태값을 변화하지 않는 상태가 된다는 의미이다.

그렇다면 어떤 neuron $n_i$의 state 값을 0에서 1로 혹은 1에서 0으로 바꿨을 때, global energy는 얼마나 변하는지를 표현하는 **energy gap**에 대해 생각해 보자. energy gap은 global energy function을 $s_i$에 대해 미분하거나, 말 그대로 $s_i=0$일때의 global energy $E(s_i=0)$과 $s_i=1$일때의 global energy $E(s_i=1)$의 차를 계산하면 아래와 같은 표현을 얻을 수 있다.

$$Energy ~ gap = \Delta E_i = E(s_i=0) - E(s_i=1) = b_i + \sum_j s_j w_{ij}$$

state update을 위해 무작위로 하나의 neuron을 선택한 후, (현재의 state값과는 무관하게) 이 neuron의 상태값을 1로 할지 0으로 할지를 결정해야 한다. 만약 energy gap이 양수라면 global energy를 줄이기 위해서는 상태값을 1로 바꿔야 할 것이고, 음수라면 0으로 바꿔야 할 것이다. 흥미롭게도 이런 결정은 **binary threshold decision rule**과 같고 그 표현($b_i + \sum_j s_j w_{ij}$)또한 유사하다.

즉 어떤 neuron $n_i$와 연결된 neuron들 중 state값이 1인 neuron과의 connection weight들과 neuron $n_i$의 bias $b_i$를 더한 값이 0보다 클 경우 $n_i$의 state을 1로 하고, 그렇지 않을 경우 0으로 하면 global energy가 감소(혹은 변하지 않게)하게 된다.

<u>Setting to an energy minimum</u>

bias와 weight값이 정해져 있을 때, global energy를 최소로하는 state값들을 찾아보자. 우선 random하게 state을 할당하여 시작점으로 잡고, random하게 하나의 neuron을 선택하고 이 neuron의 state값을 0으로 할지 1로 할지 결정한다. random한 neuron선택과 state update을 반복하다보면 더 이상 state값이 변하지 않는 상태가 오는데, 이때의 global energy 값이 energy minimum이라 할 수 있다.

<u>A deeper energy minimum</u>

그런데 초기에 할당한 random states에 따라 energy minimu값과 이때의 states가 달라질 수 있다. 예를들어 아래 그림에서 첫번째 states일 때에는 energy minimum값이 4이고 더 이상 state update이 되지 않는다. 반면 아래 두번째 그림의 경우 energy minimum 값이 5가 된다. 

<img  src="./_images/11_hopfield_vs1.PNG"/>

<img  src="./_images/11_hopfield_vs2.PNG"/>

<u>Why do the decisions need to be sequential?</u>

각 neuron의 state값을 변화시키는 방법은 비동기(asynchronous) 방식으로 한번에 하나의 neuron의 state값을 바꾸는 방법과, 동기(synchronous) 방식으로 한번에 모든 neuron의 state값을 바꾸는 방법이 있다. 그런데 synchronous하게 state을 update할 경우 각 neuron에서는 energy를 줄이려는 update을 하지만 global energy는 오히려 증가하거나 oscillate할 수 있다. 따라서 sequential하게(=asynchronous) state을 update한다.

<u>A neat way to make use of this type of computation</u>

1982년 Hopfield는 'Neural networks and physical systems with emergent collective computational abilities'라는 논문에서 evergy based model을 이용하여 값들을 기억시킬 수 있다고 제안했다. 즉 symmetric weight들을 갖는 neural net에서 energy minima가 되는 NN의 상태를 어떤 기억된 값 처럼 사용할 수 있다는 것이다. 

이런 특성을 이용하면 어떤 기억의 일부만을 이용해 기억 전체를 얻을 수 있는 'content-addressable memory'와 같은 시스템을 구현할 수 있게 된다. 이런 아이디어가 16BG에 제안됐다는 점이 참 놀랍다.(16 years before google came out) 또한 이런 network에서는 일부 unit들을 제거하더라도 시스템의 성능이 크게 손상되지 않는 좋은 특성을 갖는다.

<u>Storing memories in a Hopfield net</u>

어떤 neuron $n_i$가 random하게 선택되었고, energy를 감소시키는 $s_i$값을 결정해야 한다고 해 보자. 이때 $n_i$와 연결된 neuron이 $n_j$라하면, $w_{ij}$이 양수이면 $s_i$값이 $s_j$값으로 attract되고, 음수이면 repel된다. 

$$Energy ~ gap = \Delta E_i = E(s_i=0) - E(s_i=1) = b_i + \sum_j s_j w_{ij}$$

energy gap이 위와 같으므로 $w_{ij}$가 양수일 경우, energy gap은 $s_j$만큼 증가((음수일 경우 감소)한다.  따라서 $w_{ij}$가 양수일 경우 $s_i$값은 $s_j$값에 가까워질(attracted) 가능성이 커진다. 반면 $w_{ij}$가 음수일 경우 반대의 상황이 벌어진다.

이런 특성을 이용면 weight값을 적절히 설정하여 Hopfield net을 binary state vector를 저장하도록(기억하도록) 만들 수 있다. 

activity(state value)값이 -1과 1일 경우, weight값을 아래와 같이 설정하고
$$\Delta w_{ij} = s_i s_j$$   

activity(state value)값이 1과 0일 경, weight값을 아래와 같이 설정한다.
$$\Delta w_{ij} = 4(s_i - \frac{1}{2})(s_j - \frac{1}{2})$$

### Lecture 11b, Dealing with spurious minima

<u>The storage capacity of a Hopfield net</u>

Hopfield's storage rule에 따르면 N개의 unit으로 구성된 Hopfield net으로는 최대 0.15N개의 state vector만을 저장할 수 있다.

<u>Spurious minima limit capacity</u>

state update을 계속하다보면 global energy가 어떤 energy minimum값으로 가게 된다.(attracted) 그리고 이때의 state값들이 어떤 기억된 값으로 사용되는데, 유사한 minima가 있을 경우 정확히 원하는 state값들이 복원되는 것이 아니라 겹쳐진 잘못된 state값이 나오는 경우가 발생한다. 이런 minimum들을 spurious minima라고 한다. 이런 spurious minima 때문에 Hopfield net의 storage capacity가 일정 크기로 제한된다.

<img  src="./_images/11_hopfield_spurious.PNG"/>

<u>Avoiding spurious minima by unlearning</u>

Spurious minima를 제거하기 위해 unlearning(=<a href="https://en.wikipedia.org/wiki/Reverse_learning">reverse learning</a>)을 제거할 수 있다. (이 과정은 REM sleep시 일어나는 일 이라는 연구가 있음)

### Lecture 11c, Hopfield nets with hidden units

<u>A different computational role for Hopfield nets</u>

어떤 정보를 저장하는데 hopfield network를 사용하는 대신    
입력 데이터의 규칙 혹은 해석을 표현하는데 사용할 수도 있다.

아래의 그림에서 하단의 input data가 'visible units'로 들어오고, 이것들이 상단의 hopfield net(hidden units)로 전달된다. hopfield net은 state update을 거쳐 어떤 energy minimum으로 안정화 되는데, 이때의 state값들이 바로 input data에 대한 규칙 혹은 해석으로 사용된다. 만약 hopfield net의 energy가 비교적 작다면 그때의 state vector는 input data에 대한 좋은 해석이라 할 수 있고, 크다면 그 반대의 경우가 된다.

<img  src="./_images/11_hopfield_hidden.PNG"/>

<u>What can we infer about 3-D edges from 2-D lines in an image?</u>

3차원 사물에 대한 2차원 이미지로 표현한다고 했을 때, 3차원 이미지의 많은 정보가 유실된다. 관측되는 2차원 이미지들만으로 3차원 사물에 대한 정보를 복원해낼 수 있을까?

<img  src="./_images/11_2d_3d.PNG"/>

<u>An example: Interpreting a line drawing</u>

3차원 공간상의 6면체가 있다고 했을때, 우리가 시각적으로 관측하는 것은 2차원 공간상의 여러 선이다. 눈에 비친 어떤 선은 실제로 여러 3차원 공간의 선의 투영이라고 할 수 있다. 이런 투영 정보를 토대로 3차원 사물의 형태를 재구성 하는 문제를 NN 형태로 풀어볼 수 있다.

아래와 같이 하나의 2-D line은 여러 3-D line 중 하나가 투영된 결과라고 할 수 있고, 실제 3-D line은 서로 다른 3-D line과 연결되어 있을 것이다. 2차원 정보가 쌓여가면 실제 존재하는 연결의 경우 보다 강한 edge로 표현할 수 있을 것이다. 

<img  src="./_images/11_3d.PNG"/>

<u>Two difficult computational issues</u>

hidden unit들의 상태를 이용해 input data에 대한 어떤 규칙이나 해석을 얻어내는데에는 몇가지 난관이 있다.

우선 energy function이 poor local minima에 빠지는 문제가 있고, connection weight들을 어떻게 학습/선택할 것인가의 문제가 있다. 첫번째 문제는 이 장에서 두번재 문제는 Week12에서 살펴본다.

### Lecture 11d, Using stochastic units to improve search

energy function이 local minima에 빠지는 문제에 대한 해결책으로 적절한 noise를 더하는 방법을 생각해 볼 수 있다.

<u>Noisy networks find better energy minima</u>

'Lecture 11a'에서 hopfield net의 state update을 진행할 때, global energy function의 값이 줄어드는 방향으로만 state값을 바꿔나간다. 그런데 이렇게 할 경우 local minima에 빠지는 문제를 피할 수 없다. 

이에 대한 해결책으로 random noise를 state update에 사용하는 방법이 있다. 즉 noise를 더할 경우 다소 global energy가 증가하더라도 아래 그림에서 local minima A 빠져나와 minima B를 찾아갈 수 있다는 것이다. noise를 사용할 때 초반에는 다소 큰 noise를 부여해서 많은 energy space를 탐색할 수 있도록 하고 점차 noise를 줄여나가는 방법이 효과적이라고 알려져 있다. 이런 방법을 **simulated annealing**(어닐링)이라 한다. (Kirkpatrick, 컬크 패트릭이 1981년에 제안함)

<img  src="./_images/11_local_minima.PNG"/>

<u>How temperature affects transition probabilities</u>

energy minima를 찾아갈 때 noise를 사용하는 기법을 물리학적 관점에서 아래와 같이 생각해볼 수 있다.

어떤 입자가 A와 B에 있을 때, 더 높은 지점인 A에서 더 낮은 지점인 B로 이동하는 확률은 그 반대의 경우에 비해 높을 것이다. 그런데 온도가 높은 상황에서는 입자들의 운동이 활발할 것이기 때문에 그 이동 확률이 높을 것이고, 입자들이 B로 모여드는데 걸리는 시간이 상대적으로 짧을 것이다. 반면 온도가 낮은 상황에서는 입자가 A-B 사이를 이동하는 확률은 상대적으로 낮고 입자들이 B에 모여드는데 걸리는 시간이 더 길 것이다. 또한 온도가 낮은 상황에서의 $\frac{P(A \rightarrow B)}{P( A \leftarrow B)}$ 가 월등히 크기 때문에 입자의 이동이 제한된다. 

이런 특성을 종합해 보면 단시간 내에 입자들이 낮은 지점에 모이게 하기 위해서는   
입자의 밀집도가 낮은 초반에는 온도를 높게 하고 점차 온도를 낮게 하는 것이 효과적일 것이라는 추론이 가능하다.

<img  src="./_images/11_temp.PNG"/>

<u>Stochastic binary units</u>

[Lecture 11a - The energy function]에서 state을 update할 때 binary threshold decision rule을 사용한다고 했었다. 아래의 energy gap을 계산해서 그 값이 0보다 클 경우 neuron $n_i$의 state value $s_i$를 1로 바꿨었고, 0보다 작을 경우 0으로 바꿨었다.

$$Energy ~ gap = \Delta E_i = E(s_i=0) - E(s_i=1) = b_i + \sum_j s_j w_{ij}$$

noise를 사용할 경우 확률적으로 이런 update를 진행한다. 즉 energy gap이 클 경우 1로 바꿀 확률이 높아지고, energy gap이 작을 경우 0으로 바꿀 확률이 높아진다. 여기에 energy gap에 얼마나 민감하게 그 확률이 커지거나 작아지는지를 표현하기 위해 temperature와 같은 상수를 추가로 사용한다. 즉 $s_i$를 1로 바꿀 확률을 아래와 같이 사용한다.

$$p(s_i = 1) = \frac{1}{1 + e^{-\Delta E_i ~/~ T}}$$

위 식은 단순히 logistic function인데 T가 커질 경우 energy gap의 영향이 줄어들고, T가 극단적으로 커질 경우 $e^{-\Delta E_i ~/~ T}$값은 1에 가까워져 $p(s_i = 1)$가 $1/2$가 된다.

반면 $T=0$일 경우 단순히 energy gap의 부호에 따라, 양수일 경우 1로 음수일 경우 0으로 $s_i$값을 바꾸게 되어 binary threshold unit과 같아진다.

<u>Simulated annealing is a distraction</u>

앞서 소개한 Simulated annealing 기법을 사용하면 energy function이 local otima에 빠지는 현상을 개선하는 효과가 있다. 또한 이런 아이디어가 Terry Sejnowski(John Hopfield의 제자)와 Geoffery hinton이 Boltzmann machines을 고안하는데 영향을 준 것도 사실이다. 하지만 Boltzmann machine을 이해하는데 있어서 큰 방해가 되기 때문에 앞으로의 논의에 있어서는 단순히 $T=1$인 stochastic binary unit(단순히 energy gap을 입력으로하는 logistic function)만을 사용할 것이다.

<u>Thermal equilibrium at a temperature of 1</u>, <u>Approaching thermal equilibrium</u>

Boltzmann machine에서의 learning procedure를 이해하기 위해 알아야 하는 개념중 하나가 **thermal equilibrium(평형 상태)**이다. 이 논의에서는 온도를 $T=1$이라 가정할 것이기 때문에 'thermal equilibrium at a fixed temperature'에 대해 알아볼 것이다.

thermal equilibrium에 도달했다는 것을 그 시스템의 energy가 어떤 minimum에 도달해서 시스템 configuration(states of neuron)이 고정된 값을 갖게 된다고 생각하기 쉬운데, 이는 잘못된 표현이고, configuration에 대한 probability distribution(확률 분포)이 변하지 않는 상태가되었다는 것이 옳은 표현이다. 즉 어떤 stationary distribution(정상 분포, 확률 분포가 특정 분포에 도달하여 변하지 않을 때의 그 분포)에 도달했다는 것이다. 실제 thermal equilibrium에 도달 했다 하더라도 (온도가 0이 아닌 이상) 각 unit의 state은 여전히 변하고 있는 혹은 변할 수 있는(rattling around) 상태이다.

좀더 쉬운 예를 들어보자면, stochastic binary unit을 사용하는 똑같은 초기 구성의 hopfield net이 100개 있다고 하자. 그리고 시간이 지나면서 각 network의 unit들의 state값은 다르게 변하게 될 텐데, 시간 $t$가 지났을 때 특정 configuration에 대한 확률은 100개의 network 중 그 configuration을 갖는 network의 수이다. 이후 시간이 지나더라도 각 configuration을 갖는 network의 수가 변하지 않는다면 thermal equilibrium에 도달했다고 할 수 있다. 물론 이 상태에서도 각 network의 state은 변할 수 있지만 특정 configuration을 갖는 network의 수는 변하지 않는다.

<u>An analogy</u>

thermal equilibrium를 card shuffling에 비유해 보자면...   

Las Vegas의 모든 card dealer들이 모였다고 해 보자.(그런데 52!보다는 많은 수의 dealer들이 모여야 함) 모두 동일하게 같은 순서로 쌓여 있는 52장의 카드 덱을 받는다. 카드가 섞여 있는 가지 수는 $52!$인데, 초기에는 100명의 card dealer들이 모두 특정 순서의 카드 덱을 갖고 있다.

이제 모두 같이 카드를 섞기 시작하는데, 당연히 모두 조금은 다른 방식으로 카드를 섞을 것이고 혹은 우연에 의해 카드의 순서가 딜러마다 달라질 것이다. 그러나 카드를 섞기 시작한 초반에는 같은 순서를 갖는 카드 덱을 들고 있는 dealer들이 상당수 있을 것이다. 하지만 시간이 지나면서 $52!$가지의 순서의 빈도가 점차 동일해 질 것이다. 이런 상태가 바로  thermal equilibrium이다. 이 상태가 되었어도 각 dealer의 card 순서는 계속 바뀔 수 있지만 덱 순서의 uniform distribution에서 거의 변하지 않게 된다. (이 비유에서의 가정은 모든 카드 덱 순서의 energy는 동일하다는 것이다.)

### Lecture 11e, How a Boltzmann machine models data

앞서 마지막에 다뤘던 Stochastic Hopfield nets with hidden units이 바로 **Boltzmann machines**인데, Boltzmann machine을 이용해 binary data vector를 모델링하는 방법에 대해 배워볼 것이다. 또한 왜 binary data vector를 모델링하려는지와 이것을 이용해서 무엇을 할 수 있는지에 대해 알아볼 것이다. 마지막으로 Boltzmann machine에서 각 binary vector의 확률 분포가 각 weight과 어떤 관계가 있는지 알아볼 것이다.

<u>Modeling binary data</u>

binary vector로 이루어진 training set 데이터를 이용해서, 모든 가능한 binary vector에 대한 확률 분포가 나오도록 모델 fitting을 할 수 있다고 해 보자.

어떤 특정 분포에서 나온 binary vector들이 있고, 이 데이터를 이용해 binary vector에 대한 확률 분포를 만들었다고 하자. 이제 새로운 획득한 binary vector가 있다면 이 binary vector가 앞서의 확률 분포에서 나왔을 확률을 알 수 있다. (즉 동일 분포에서 나온 데이터인지 판단할 수 있게 된다.)

혹은 특이 상황에 대한 탐지에도 이런 모형을 사용할 수 있다. 예를들어 발전소의 모든 탐지 데이터가 binary 값으로 이루어져 있다면, 발전소의 특정 상황을 binary vector로 나타낼 수 있을 것이다. 이상 현상에 대한 데이터가 준비되어 있지 않더라도 보통 상황에서의 binary vector들을 이용해 이들의 확률 분포를 구할 수 있고, 어떤 특이상황이 발생했을 경우 이 새로운 binary vector에 대한 확률이 현저하게 낮을 것이기 때문에 특이상황이라는 것을 탐지할 수 있게 된다.

또 다른 예로 여러 가지의 모델(확률 분포)가 있을 때, 새로 획득한 binary vector가 어떤 모델에서 나온 것인 알고 싶을 수 있다. 이 경우 아래와 같이 bayes theorem을 이용해 새로운 데이터가 주어졌을 때 특정 모델이 정답일 확률을 계산할 수 있다.

$$p(Model ~ i ~ | ~ data) = \frac{p(data ~ | ~ Model ~ i)}{\sum_j p(data ~ | ~ Model ~ j )}$$

<u>(1). How a causal model generated data</u>

어떤 데이터에 대한 (확률)모델을 만들어내는 두가지 방법에 대해 소개하려 한다. 우선 **causal model**을 이용하는 방법인데, 두 단계에 걸쳐 확률적으로 데이터를 발생시킨다.

<img  src="./_images/11_causal2.PNG"/>

우선 위 그림에서 hidden으로 표시된 layer와 같이 어떤 latent variable(잠재변수)가 존재하고, 이들의 binary state value는 각 hidden unit의 prior distribution에서 추출하여 결정한다.(보통 각 hidden unit의 prior distribution들을 서로 독립임을 가정한다.)

다음으로 hidden layer와 연결된 visible layer의 unit들의 상태값을 확률적으로 결정한다. (여기에서는 stochastic logistic unit을 사용하고 있음)

이런식으로 visible layer의 unit들의 state value를 결정할 수 있다고 할 때, 각 visible layer의 state vector에 대한 확률은 아래와 같이 계산할 수 있다.

$$p(\mathbf{v}) = \sum_h p(\mathbf{h}) ~ p( \mathbf{v} ~ | ~ \mathbf{h})$$

<u>(2). How a Boltzmann Machine generates data</u>

어떤 데이터에 대한 (확률)모델을 만들어내는 두번째 방법은 **Boltzmann machine**을 사용하는 것이다. 이 방법은 causal generative model이 아닌고, 대신 visible unit들과 hidden unit들의 joint configuration(모든 unit에 대한 확률이라 생각하면 됨)에 대한 energy를 이용하여 확률 분포를 정의한다.

joint configuration에 확률 분포를 energy를 이용하여 정의하는 두가지 접근법이 있다.
- joint configuration에 확률 분포를 아래와 같이 정의하는 방법
$$p(\mathbf{v}, \mathbf{h}) \propto e^{-E(\mathbf{v},\mathbf{h})} $$
- 수없이 모든 stochastic binary unit들을 update 한 후 특정 joint configuration이 나타날 확률

그런데 위 두 가지 정의는 사실 동일한 의미이다.

<u>The Energy of a joint configuration</u>

앞서 Hopfield network에서 global energy를 정의 했던 것과 같이, boltzmann machine에서도 유사하게 아래와 같이 energy function을 정의할 수 있다. (hidden과 visible이 나뉘고 각 unit간의 상호 작용을 표현하는 방식은 hopfield network에서의 그것과 일맥상통하다.)

<img  src="./_images/11_energy_joint_conf.PNG"/>

<u>Using energies to define probabilities</u>

앞서 joint configuration에 대한 energy function을 정의 했었고, 이제 이것을 이용해 joint configuration에 대한 probability를 정의해 보자.

특정 joint configuration에 대한 확률은      
특정 joint configuration에 대한 energy를 모든 경우의 joint configuration의 energy를 더한 것으로 정의 된다. (분모항은 nomalization term인데 물리학자들은 partition function이고 부른다고 한다.)

<img  src="./_images/11_bm_energy.PNG"/>

visible과 hidden의 joint probability를 구했으니, 이를 이용해 visible unit들에 대한 확률 분포는 아래와 같이 구할 수 있다.(h에 관한 모든 항을 더함)

<img  src="./_images/11_bm_prob.PNG"/>

아래와 같은 Boltzmann machine이 주어졌을 때, 각 visible binary vector에 대한 확률을 구해보도록 하자.

<img  src="./_images/11_bm_example.PNG"/>

우선 visible binary vector는 [1,1], [1,0], [0,1], [0,0]의 4가지가 있고, 아래 표에서 각각 다른 색으로 표시되어 있다. 다음으로 이런 visible binary vector가 주어졌을 때 hidden layer의 binary vector는 동일하게 4가지가 가능하고 각각의 경우 -energy는 아래 표의 $-E$와 같이 계산할 수 있다. 예를들어 첫번째 줄의 -energy는 $2=2-1+1$와 같다. 다음으로 $e^{-E}$(un-normalized probability)를 구하고, 각 $\mathbf{v}$에 대한 $e^{-E}$의 비율을 계산하여 $p(\mathbf{v}, \mathbf{h})$를 구하게 된다. 다음으로 이 값들을 더해 $p(\mathbf{v})$를 구할 수 있다.

<img  src="./_images/11_bm_table.PNG"/>

<u>Getting a sample from the model</u>

그런데 hidden unit의 수가 많을 경우 앞서와 같이 normalizaing term($\sum_{\mathbf{u}, \mathbf{g}} e^{-E(\mathbf{u},\mathbf{g})}$)을 계산하는 것이 불가능하게 된다. (모든 경우에 대한 energy를 계산하는 것이 어려워짐, 즉 $e^{E}$을 알고 있더라도 $p(\mathbf{v}, \mathbf{h})$를 구하기 어려워짐)

이런 문제를 해결하기 위해 **Markov Chain Monte Carlo**방법을 사용한다.
- 우선 random하게 global configuration을 설정하고
- random하게 특정 unit을 골라 energy gap에 따라 stochastically update한다.
- (이런 선택과 update을 어느정도 반복하고)
- 어느 시점에서의 global configuration을 sampling하고, 이때의 energy를 계산한다. $p(\mathbf{v}, \mathbf{h}) \propto e^{-E(\mathbf{v}, \mathbf{v})}$이므로 그때의 확률을 계산할 수 있고
- 이런 update과 sampling을 계속하여, 확률 분포가 stationary distribution에 이르면 sampling을 멈춘다.

즉 이 과정을 통해 $p(\mathbf{v}, \mathbf{h}) \propto e^{-E(\mathbf{v}, \mathbf{h})}$를 구하게 된다. (다음으로 $\mathbf{h}$에 대해 marginalize하여 $p(\mathbf{v})$구할 수 있음)

<u>Getting a sample from the posterior distribution over hidden configurations for a given data vector</u>

앞서 [Getting a sample from the model]에서는 모든 configuration을 random으로 설정하고 MCMC(Markov Chain Monte Carlo) 방법을 사용 했었다. 그런데 이런 경우 hidden configuration의 수가 많을 수 있기 때문에, visible layer는 given data로 고정한 상태에서 MCMC를 진행하는 방법을 사용할수도 있다. 이때 visible layer가 주어졌기 때문에 이는 posterior distribution($p(h|v)$)에서 추출하는 것이다.

이런 방식을 weight들을 학습할 때 유용하게 사용될 수 있는데, 각각의 hidden configuration은 observed data에 대한 설명(explanation)이라 할 수 있다. 즉 weight을 조절하여 energy가 더 낮을 경우 더 나은 설명이 되는 것이다.

.